# **Merge Data ISPU dari tahun 2010-2025 menjadi 1 file csv**

## Import Library

In [1]:
import pandas as pd
import numpy as np

## Files and defining Target Column

In [ ]:
files = [
    "../../data/ISPU/indeks-standar-pencemaran-udara-(ispu)-tahun-2010-komponen-data.csv",
    "../../data/ISPU/indeks-standar-pencemaran-udara-(ispu)-tahun-2011-komponen-data.csv",
    "../../data/ISPU/indeks-standar-pencemaran-udara-(ispu)-tahun-2012-komponen-data.csv",
    "../../data/ISPU/indeks-standar-pencemaran-udara-(ispu)-tahun-2013-komponen-data.csv",
    "../../data/ISPU/indeks-standar-pencemaran-udara-(ispu)-tahun-2014-komponen-data.csv",
    "../../data/ISPU/indeks-standar-pencemaran-udara-(ispu)-tahun-2015-komponen-data.csv",
    "../../data/ISPU/indeks-standar-pencemaran-udara-(ispu)-tahun-2016-komponen-data.csv",
    "../../data/ISPU/data-indeks-standar-pencemar-udara-(ispu)-di-provinsi-dki-jakarta-2023-komponen-data.csv",
    "../../data/ISPU/data-indeks-standar-pencemar-udara-(ispu)-di-provinsi-dki-jakarta-komponen-data-2024.csv",
    "../../data/ISPU/data-indeks-standar-pencemar-udara-(ispu)-di-provinsi-dki-jakarta-komponen-data-2025.csv"
]

target_columns = [
    'periode_data', 'tanggal', 'stasiun', 
    'pm_sepuluh', 'pm_duakomalima', 
    'sulfur_dioksida', 'karbon_monoksida', 'ozon', 'nitrogen_dioksida', 
    'max', 'parameter_pencemar_kritis', 'kategori'
]

## Uniformity of Column Name

In [3]:
rename_mapping = {
    'pm10': 'pm_sepuluh',
    'so2': 'sulfur_dioksida',
    'co': 'karbon_monoksida',
    'o3': 'ozon',
    'no2': 'nitrogen_dioksida',
    'critical': 'parameter_pencemar_kritis',
    'categori': 'kategori',
    'lokasi_spku': 'stasiun',
    'pm_sepuluh': 'pm_sepuluh',
    'pm_duakomalima': 'pm_duakomalima'
}

## Rename Column

In [4]:
data_list = []

for filename in files:
    try:
        df = pd.read_csv(filename)
        df.columns = df.columns.str.lower().str.strip()
        df = df.rename(columns=rename_mapping)

        if 'tanggal' in df.columns:
            df['tanggal'] = df['tanggal'].astype(str).str.strip()
            df['tanggal'] = df['tanggal'].replace(['---', '', 'nan'], np.nan)

            mask_valid = df['tanggal'].notna()
            
            if mask_valid.any():
                sample_date = df.loc[mask_valid, 'tanggal'].iloc[0]

                if len(sample_date) <= 2 or ("-" not in sample_date and "/" not in sample_date):
                    if 'periode_data' in df.columns:
                        df['tahun_temp'] = df['periode_data'].astype(str).str[:4]
                        df['bulan_temp'] = df['periode_data'].astype(str).str[4:6]

                        df.loc[mask_valid, 'tanggal'] = (
                            df.loc[mask_valid, 'tahun_temp'] + "-" + 
                            df.loc[mask_valid, 'bulan_temp'] + "-" + 
                            df.loc[mask_valid, 'tanggal'].str.zfill(2) 
                        )

            df['tanggal'] = pd.to_datetime(df['tanggal'], dayfirst=True, errors='coerce')

        for col in target_columns:
            if col not in df.columns:
                df[col] = np.nan

        df_final = df[target_columns]
        df_final = df_final.replace(['---', ''], np.nan)
        df_final = df_final.dropna(subset=['tanggal'])
    
        data_list.append(df_final)

    except FileNotFoundError:
        print(f"File tidak ditemukan")

C:\Users\reube\AppData\Local\Temp\ipykernel_28768\9720172.py:29: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df['tanggal'] = pd.to_datetime(df['tanggal'], dayfirst=True, errors='coerce')
C:\Users\reube\AppData\Local\Temp\ipykernel_28768\9720172.py:29: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df['tanggal'] = pd.to_datetime(df['tanggal'], dayfirst=True, errors='coerce')
C:\Users\reube\AppData\Local\Temp\ipykernel_28768\9720172.py:29: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df['tanggal'] = pd.to_datetime(df['tanggal'], dayfirst=True, errors='coerce')
C:\Users\reube\AppData\Local\Temp\ipykernel_28768\9720172.py:29: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst

## Merging Dataset

In [ ]:
if data_list:
    full_df = pd.concat(data_list, ignore_index=True)
    
    cols_numeric = ['pm_sepuluh', 'pm_duakomalima', 'sulfur_dioksida', 'karbon_monoksida', 'ozon', 'nitrogen_dioksida', 'max']
    for col in cols_numeric:
        full_df[col] = pd.to_numeric(full_df[col], errors='coerce')

    full_df = full_df.sort_values('tanggal')
    full_df.to_csv('../../data/ISPU/ISPU_DKI_Gabungan.csv', index=False)

else:
    print("Tidak ada data yang berhasil digabungkan.")